In [4]:
import pandas as pd
import numpy as np
import math
from collections import Counter
from sklearn.model_selection import KFold
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    mean_squared_error,
    mean_absolute_error,
    r2_score,
)

# --- Helper Functions for Entropy/Gain/Variance ---

def calculate_entropy(y):
    """Calculates entropy of a given target array y (integer-encoded)."""
    if len(y) == 0:
        return 0
    # np.bincount efficiently counts occurrences of each integer label
    hist = np.bincount(y)
    ps = hist / len(y)
    return -np.sum([p * math.log2(p) for p in ps if p > 0])

def calculate_information_gain(y, subsets_y):
    """Calculates Information Gain."""
    total_len = len(y)
    if total_len == 0:
        return 0
    parent_entropy = calculate_entropy(y)
    
    # Calculate weighted average of child node entropies
    weighted_child_entropy = sum((len(subset) / total_len) * calculate_entropy(subset) 
                                 for subset in subsets_y if len(subset) > 0)
    
    return parent_entropy - weighted_child_entropy

def calculate_split_info(subsets_y):
    """Calculates the Split Info (Intrinsic Information) for C4.5."""
    total_len = sum(len(subset) for subset in subsets_y)
    if total_len == 0:
        return 0
    
    split_info = 0
    for subset in subsets_y:
        proportion = len(subset) / total_len
        if proportion > 0:
            split_info -= proportion * math.log2(proportion)
            
    # Add a small epsilon to avoid division by zero if split_info is 0
    return split_info + 1e-6

def calculate_gain_ratio(y, subsets_y):
    """Calculates Gain Ratio for C4.5."""
    info_gain = calculate_information_gain(y, subsets_y)
    split_info = calculate_split_info(subsets_y)
    
    # split_info will have 1e-6 added, so it's never zero
    return info_gain / split_info

def calculate_variance(y):
    """Calculates variance of a target array y (for regression)."""
    if len(y) == 0:
        return 0
    return np.var(y)

def calculate_variance_reduction(y, subsets_y):
    """Calculates Variance Reduction (like Information Gain for regression)."""
    total_len = len(y)
    if total_len == 0:
        return 0
    parent_variance = calculate_variance(y)
    
    # Calculate weighted average of child node variances
    weighted_child_variance = sum((len(subset) / total_len) * calculate_variance(subset) 
                                  for subset in subsets_y if len(subset) > 0)
    
    return parent_variance - weighted_child_variance

# --- Node and Base Tree Class ---

class Node:
    """Decision Tree Node."""
    def __init__(self, feature=None, threshold=None, children=None, *, value=None, is_leaf=False):
        self.feature = feature       # Feature index to split on
        self.threshold = threshold   # Value/threshold for continuous split
        self.children = children     # Dict of child nodes {'val': Node, ...}
        self.value = value           # Prediction value (if leaf, or majority value for pruning/unseen)
        self.is_leaf = is_leaf

class MyDecisionTreeBase:
    """Base class for 'from scratch' decision trees."""
    def __init__(self, min_samples_split=2, max_depth=100):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.root = None
        self._feature_types = []
        self._target_type = 'classification'
        self.feature_names = None
        self.label_map = {}
        self.reverse_label_map = {}
        self.default_value = None # For unseen categories in predict

    def fit(self, X, y):
        """Build the decision tree."""
        # Convert pandas to numpy
        if isinstance(X, pd.DataFrame):
            self.feature_names = X.columns
            X = X.values
        if isinstance(y, pd.Series):
            y = y.values
            
        # Determine feature types
        self._feature_types = []
        for i in range(X.shape[1]):
            # Treat as continuous if numeric and has > 10 unique values
            if pd.api.types.is_numeric_dtype(X[:, i]) and len(np.unique(X[:, i])) > 10:
                 self._feature_types.append('continuous')
            else:
                 self._feature_types.append('categorical')

        # Determine target type and encode
        if pd.api.types.is_numeric_dtype(y):
            self._target_type = 'regression'
            self.default_value = np.mean(y) # Default is mean for regression
        else:
            self._target_type = 'classification'
            self.label_map = {label: i for i, label in enumerate(np.unique(y))}
            self.reverse_label_map = {i: label for label, i in self.label_map.items()}
            y = np.array([self.label_map[label] for label in y])
            self.default_value = Counter(y).most_common(1)[0][0] # Default is majority class

        self.root = self._build_tree(X, y, depth=0)

    def _build_tree(self, X, y, depth):
        n_samples = len(y)
        
        # Get the value for this node (used for leaf or fallback)
        current_leaf_value = self._get_leaf_value(y)

        # Stopping criteria
        if (depth >= self.max_depth or
            n_samples < self.min_samples_split or
            len(np.unique(y)) == 1):
            
            return Node(value=current_leaf_value, is_leaf=True)

        n_features = X.shape[1]
        best_split = self._find_best_split(X, y, n_features)

        # If no split provides positive gain, create a leaf
        if best_split is None or best_split['gain'] <= 0:
            return Node(value=current_leaf_value, is_leaf=True)

        feature_idx = best_split['feature_idx']
        
        children = {}
        node_threshold = None
        
        if self._feature_types[feature_idx] == 'continuous':
            node_threshold = best_split['threshold']
            indices_left, indices_right = best_split['indices_left'], best_split['indices_right']
            
            left_node = self._build_tree(X[indices_left, :], y[indices_left], depth + 1)
            right_node = self._build_tree(X[indices_right, :], y[indices_right], depth + 1)
            
            children = {
                '<= threshold': left_node,
                '> threshold': right_node
            }
        
        else: # Categorical
            indices_map = best_split['indices_map']
            for value, indices in indices_map.items():
                child_node = self._build_tree(X[indices, :], y[indices], depth + 1)
                children[value] = child_node

        # Store the majority value of this node for fallback predictions
        return Node(feature=feature_idx, threshold=node_threshold, children=children, value=current_leaf_value)
    
    def _get_leaf_value(self, y):
        """Get the value for a leaf node."""
        if self._target_type == 'classification':
            most_common = Counter(y).most_common(1)
            # Use default value if y is empty
            return most_common[0][0] if most_common else self.default_value
        else:
            # Use default value if y is empty
            return np.mean(y) if len(y) > 0 else self.default_value

    def _find_best_split(self, X, y, n_features):
        """Abstract method to be implemented by subclasses."""
        raise NotImplementedError

    def predict(self, X):
        """Predict class labels or values for X."""
        if isinstance(X, pd.DataFrame):
            X = X.values
            
        predictions = [self._predict_row(x, self.root) for x in X]
        
        if self._target_type == 'classification':
            # Convert integer labels back to original labels
            default_original = self.reverse_label_map.get(self.default_value)
            return np.array([self.reverse_label_map.get(pred, default_original) for pred in predictions])
        else:
            return np.array(predictions)

    def _predict_row(self, x, node):
        """Helper to predict a single row by traversing the tree."""
        if node.is_leaf:
            return node.value

        feature_val = x[node.feature]
        
        try:
            if self._feature_types[node.feature] == 'continuous':
                if feature_val <= node.threshold:
                    return self._predict_row(x, node.children['<= threshold'])
                else:
                    return self._predict_row(x, node.children['> threshold'])
            else: # Categorical
                if feature_val in node.children:
                    return self._predict_row(x, node.children[feature_val])
                else:
                    # Handle unseen category: return the majority value of the *current* node
                    return node.value
        except Exception:
             # Fallback for any traversal error (e.g., node has no children)
             return node.value

    def print_tree(self):
        """Prints the decision tree structure."""
        if self.root is None:
            print("Tree has not been fitted.")
            return
        
        print(f"\n--- Tree Structure for {self.__class__.__name__} ---")
        # Pass feature names and target mapping to recursive helper
        self._print_recursive(self.root, indent="")
        print("-" * 30)

    def _print_recursive(self, node, indent):
        """Recursive helper for printing the tree."""
        if node.is_leaf:
            # Format the leaf value
            if self._target_type == 'classification':
                value = self.reverse_label_map.get(node.value, "N/A")
            else:
                value = f"{node.value:.4f}"
            print(f"{indent}Predict: {value}")
            return

        # Get feature name
        feature_name = self.feature_names[node.feature]
        
        # Check if continuous or categorical
        if node.threshold is not None:
            # Continuous split
            print(f"{indent}Split on: {feature_name} (Threshold: {node.threshold:.4f})")
            
            # Left child
            print(f"{indent}  If <= {node.threshold:.4f}:")
            self._print_recursive(node.children['<= threshold'], indent + "    ")
            
            # Right child
            print(f"{indent}  If > {node.threshold:.4f}:")
            self._print_recursive(node.children['> threshold'], indent + "    ")
            
        else:
            # Categorical split
            print(f"{indent}Split on: {feature_name} (Categorical)")
            
            # Sort children by value for consistent printing
            for value, child_node in sorted(node.children.items()):
                print(f"{indent}  If == {value}:")
                self._print_recursive(child_node, indent + "    ")


# --- ID3 Classifier ---
# (Assumes all features are categorical)
class MyID3(MyDecisionTreeBase):
    """ID3 Decision Tree Classifier. Assumes all features are categorical."""
    
    def fit(self, X, y):
        # ID3 requires all features to be categorical
        if isinstance(X, pd.DataFrame):
            self.feature_names = X.columns
            X = X.values
        if isinstance(y, pd.Series):
            y = y.values

        # Override feature types to all categorical
        self._feature_types = ['categorical'] * X.shape[1] 
        
        self._target_type = 'classification'
        self.label_map = {label: i for i, label in enumerate(np.unique(y))}
        self.reverse_label_map = {i: label for label, i in self.label_map.items()}
        y = np.array([self.label_map[label] for label in y])
        self.default_value = Counter(y).most_common(1)[0][0]

        self.root = self._build_tree(X, y, depth=0)

    def _find_best_split(self, X, y, n_features):
        best_gain = -1
        best_feature_idx = None
        best_indices_map = None

        # Iterate over all features
        for feature_idx in range(n_features):
            unique_values = np.unique(X[:, feature_idx])
            
            # Must have at least 2 unique values to split
            if len(unique_values) < 2:
                continue

            subsets_y = []
            indices_map = {}
            for value in unique_values:
                indices = np.where(X[:, feature_idx] == value)[0]
                if len(indices) > 0:
                    indices_map[value] = indices
                    subsets_y.append(y[indices])

            # If split results in less than 2 groups, skip it
            if len(subsets_y) < 2:
                continue

            gain = calculate_information_gain(y, subsets_y)

            if gain > best_gain:
                best_gain = gain
                best_feature_idx = feature_idx
                best_indices_map = indices_map
        
        if best_gain <= 0:
            return None

        return {
            'feature_idx': best_feature_idx,
            'threshold': None,
            'indices_map': best_indices_map,
            'gain': best_gain,
            'indices_left': None,
            'indices_right': None
        }

# --- C4.5 Classifier ---
# (Handles both continuous and categorical features)
class MyC45(MyDecisionTreeBase):
    """C4.5 Decision Tree Classifier."""

    def _find_best_split(self, X, y, n_features):
        best_gain_ratio = -1
        best_split = None

        for feature_idx in range(n_features):
            feature_type = self._feature_types[feature_idx]
            unique_values = np.unique(X[:, feature_idx])

            if len(unique_values) < 2:
                continue

            if feature_type == 'continuous':
                # Find best threshold for continuous feature
                sorted_values = np.sort(unique_values)
                thresholds = (sorted_values[:-1] + sorted_values[1:]) / 2

                for threshold in thresholds:
                    indices_left = np.where(X[:, feature_idx] <= threshold)[0]
                    indices_right = np.where(X[:, feature_idx] > threshold)[0]
                    
                    if len(indices_left) == 0 or len(indices_right) == 0:
                        continue

                    subsets_y = [y[indices_left], y[indices_right]]
                    gain_ratio = calculate_gain_ratio(y, subsets_y)

                    if gain_ratio > best_gain_ratio:
                        best_gain_ratio = gain_ratio
                        best_split = {
                            'feature_idx': feature_idx,
                            'threshold': threshold,
                            'indices_map': None,
                            'gain': gain_ratio,
                            'indices_left': indices_left,
                            'indices_right': indices_right
                        }
            
            else: # Categorical
                subsets_y = []
                indices_map = {}
                for value in unique_values:
                    indices = np.where(X[:, feature_idx] == value)[0]
                    if len(indices) > 0:
                        indices_map[value] = indices
                        subsets_y.append(y[indices])
                
                if len(subsets_y) < 2:
                    continue
                
                gain_ratio = calculate_gain_ratio(y, subsets_y)

                if gain_ratio > best_gain_ratio:
                    best_gain_ratio = gain_ratio
                    best_split = {
                        'feature_idx': feature_idx,
                        'threshold': None,
                        'indices_map': indices_map,
                        'gain': gain_ratio,
                        'indices_left': None,
                        'indices_right': None
                    }

        return best_split

# --- Decision Tree Regressor ---
# (Handles both continuous and categorical features)
class MyDecisionTreeRegressor(MyDecisionTreeBase):
    """Decision Tree Regressor."""

    def _find_best_split(self, X, y, n_features):
        best_var_reduction = -1
        best_split = None

        for feature_idx in range(n_features):
            feature_type = self._feature_types[feature_idx]
            unique_values = np.unique(X[:, feature_idx])

            if len(unique_values) < 2:
                continue

            if feature_type == 'continuous':
                sorted_values = np.sort(unique_values)
                thresholds = (sorted_values[:-1] + sorted_values[1:]) / 2

                for threshold in thresholds:
                    indices_left = np.where(X[:, feature_idx] <= threshold)[0]
                    indices_right = np.where(X[:, feature_idx] > threshold)[0]
                    
                    if len(indices_left) == 0 or len(indices_right) == 0:
                        continue

                    subsets_y = [y[indices_left], y[indices_right]]
                    var_reduction = calculate_variance_reduction(y, subsets_y)

                    if var_reduction > best_var_reduction:
                        best_var_reduction = var_reduction
                        best_split = {
                            'feature_idx': feature_idx,
                            'threshold': threshold,
                            'indices_map': None,
                            'gain': var_reduction, # Store var reduction as 'gain'
                            'indices_left': indices_left,
                            'indices_right': indices_right
                        }
            
            else: # Categorical
                subsets_y = []
                indices_map = {}
                for value in unique_values:
                    indices = np.where(X[:, feature_idx] == value)[0]
                    if len(indices) > 0:
                        indices_map[value] = indices
                        subsets_y.append(y[indices])
                
                if len(subsets_y) < 2:
                    continue
                
                var_reduction = calculate_variance_reduction(y, subsets_y)

                if var_reduction > best_var_reduction:
                    best_var_reduction = var_reduction
                    best_split = {
                        'feature_idx': feature_idx,
                        'threshold': None,
                        'indices_map': indices_map,
                        'gain': var_reduction,
                        'indices_left': None,
                        'indices_right': None
                    }

        return best_split


# --- Evaluation Function ---

def evaluate_model(model_class, X_df, y_s, task_type='classification', max_depth=5):
    """
    Performs 5-fold CV and prints metrics.
    Takes pandas DataFrame X_df and pandas Series y_s for easier data handling.
    """
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    
    scores = []
    
    for train_index, test_index in kf.split(X_df):
        # Use .iloc to select rows based on integer index
        X_train_df = X_df.iloc[train_index]
        y_train_s = y_s.iloc[train_index]
        X_test_df = X_df.iloc[test_index]
        y_test = y_s.iloc[test_index]
        
        model = model_class(max_depth=max_depth) # Instantiate new model
        
        # Special preprocessing for ID3
        if isinstance(model, MyID3):
            # Create copies to avoid SettingWithCopyWarning
            X_train_id3 = X_train_df.copy()
            X_test_id3 = X_test_df.copy()
            
            for col in X_train_id3.columns:
                if pd.api.types.is_numeric_dtype(X_train_id3[col]):
                    # Discretize based on *training* median
                    median = X_train_id3[col].median()
                    X_train_id3[col] = (X_train_id3[col] > median).astype(str)
                    # Apply the same median to the test set
                    X_test_id3[col] = (X_test_id3[col] > median).astype(str)
            
            model.fit(X_train_id3, y_train_s)
            y_pred = model.predict(X_test_id3)
        
        else:
            # For C4.5 and Regressor, pass data as-is
            model.fit(X_train_df, y_train_s)
            y_pred = model.predict(X_test_df)

        # Calculate metrics
        fold_scores = {}
        if task_type == 'classification':
            fold_scores['accuracy'] = accuracy_score(y_test, y_pred)
            fold_scores['precision'] = precision_score(y_test, y_pred, average='weighted', zero_division=0)
            fold_scores['recall'] = recall_score(y_test, y_pred, average='weighted', zero_division=0)
            fold_scores['f1'] = f1_score(y_test, y_pred, average='weighted', zero_division=0)
        else: # Regression
            fold_scores['mse'] = mean_squared_error(y_test, y_pred)
            fold_scores['rmse'] = np.sqrt(fold_scores['mse'])
            fold_scores['mae'] = mean_absolute_error(y_test, y_pred)
            try:
                # R2 can fail if y_test is constant
                fold_scores['r2'] = r2_score(y_test, y_pred)
            except ValueError:
                 fold_scores['r2'] = -np.inf 
        
        scores.append(fold_scores)

    # Calculate and print average scores
    avg_scores = {}
    print(f"Average Metrics (5-Fold CV) for {model_class.__name__} (max_depth={max_depth}):")
    for metric in scores[0].keys():
        avg_scores[metric] = np.mean([s[metric] for s in scores])
        print(f"  - Avg. {metric.upper()}: {avg_scores[metric]:.4f}")
    print("-" * 30)

# --- Main Execution ---

try:
    # --- Task 1: playCricket.csv ---
    print("--- Task 1: playCricket.csv Evaluation (max_depth=3) ---")
    df_play_cricket = pd.read_csv("playCricket.csv").drop(columns=['Day'])
    X_cricket = df_play_cricket.drop(columns=['PlayCricket'])
    y_cricket = df_play_cricket['PlayCricket']
    
    # Run evaluation
    evaluate_model(MyID3, X_cricket, y_cricket, task_type='classification', max_depth=3)
    evaluate_model(MyC45, X_cricket, y_cricket, task_type='classification', max_depth=3)
    
    # Train on full data and print tree
    print("\nTraining on full playCricket.csv to show tree structure:")
    id3_cricket = MyID3(max_depth=3)
    id3_cricket.fit(X_cricket, y_cricket)
    id3_cricket.print_tree()
    
    c45_cricket = MyC45(max_depth=3)
    c45_cricket.fit(X_cricket, y_cricket)
    c45_cricket.print_tree()

    # --- Task 2: drug_200.csv ---
    print("\n--- Task 2: drug_200.csv Evaluation (max_depth=4) ---")
    df_drug = pd.read_csv("drug_200.csv")
    X_drug = df_drug.drop(columns=['Drug'])
    y_drug = df_drug['Drug']
    
    # Using max_depth=4 as a compromise for printing
    evaluate_model(MyID3, X_drug, y_drug, task_type='classification', max_depth=4)
    evaluate_model(MyC45, X_drug, y_drug, task_type='classification', max_depth=4)
    
    # Train ID3 on full data and print tree
    print("\nTraining on full drug_200.csv to show tree structure (max_depth=4):")
    id3_drug = MyID3(max_depth=4)
    # Preprocess for ID3
    X_drug_id3 = X_drug.copy()
    for col in X_drug_id3.columns:
        if pd.api.types.is_numeric_dtype(X_drug_id3[col]):
            median = X_drug_id3[col].median()
            X_drug_id3[col] = (X_drug_id3[col] > median).astype(str)
    id3_drug.fit(X_drug_id3, y_drug)
    id3_drug.print_tree()

    # Train C4.5 on full data and print tree
    c45_drug = MyC45(max_depth=4)
    c45_drug.fit(X_drug, y_drug)
    c45_drug.print_tree()


    # --- Task 3: petrol_consumption.csv ---
    print("\n--- Task 3: petrol_consumption.csv Evaluation (max_depth=3) ---")
    df_petrol = pd.read_csv("petrol_consumption.csv")
    X_petrol = df_petrol.drop(columns=['Petrol_Consumption'])
    y_petrol = df_petrol['Petrol_Consumption']
    
    evaluate_model(MyDecisionTreeRegressor, X_petrol, y_petrol, task_type='regression', max_depth=3)
    
    # Train on full data and print tree
    print("\nTraining on full petrol_consumption.csv to show tree structure (max_depth=3):")
    reg_petrol = MyDecisionTreeRegressor(max_depth=3)
    reg_petrol.fit(X_petrol, y_petrol)
    reg_petrol.print_tree()

except FileNotFoundError as e:
    print(f"Error loading file: {e}")
    print("Please ensure 'playCricket.csv', 'drug_200.csv', and 'petrol_consumption.csv' are available.")
except Exception as e:
    print

--- Task 1: playCricket.csv Evaluation (max_depth=3) ---
Average Metrics (5-Fold CV) for MyID3 (max_depth=3):
  - Avg. ACCURACY: 0.9333
  - Avg. PRECISION: 0.9667
  - Avg. RECALL: 0.9333
  - Avg. F1: 0.9333
------------------------------
Average Metrics (5-Fold CV) for MyC45 (max_depth=3):
  - Avg. ACCURACY: 0.6000
  - Avg. PRECISION: 0.6222
  - Avg. RECALL: 0.6000
  - Avg. F1: 0.5733
------------------------------

Training on full playCricket.csv to show tree structure:

--- Tree Structure for MyID3 ---
Split on: Outlook (Categorical)
  If == Overcast:
    Predict: Yes
  If == Rain:
    Split on: Wind (Categorical)
      If == Strong:
        Predict: No
      If == Weak:
        Predict: Yes
  If == Sunny:
    Split on: Humidity (Categorical)
      If == High:
        Predict: No
      If == Normal:
        Predict: Yes
------------------------------

--- Tree Structure for MyC45 ---
Split on: Outlook (Categorical)
  If == Overcast:
    Predict: Yes
  If == Rain:
    Split on: Wind 